# Create the features DF
* using by_postal_code dataset

# Import

In [82]:
import pandas as pd
import json
import numpy as np
from collections import Counter
import pickle
import math 
from sklearn.model_selection import train_test_split
import torch

In [83]:
RANDOM_STATE = 24

In [84]:
# 導入資料
train_df = pd.read_pickle('../data/Train_by_postoal_code_without_review_pointwise_v3_4.pkl').reset_index(drop=True)
test_df = pd.read_pickle('../data/Test_by_postoal_code_without_review_pointwise_v3_4.pkl').reset_index(drop=True)
all_df = pd.read_pickle('../Data/restaurant_only_s_with_embedding.pkl')

In [ ]:
print(train_df.shape, test_df.shape, all_df.shape)

In [ ]:
all_df.head()

## Concat by restaurant and location 

In [ ]:
# Example list of sets
# sets_list = list(train_df[:5].LDA_res)
# combine_sets(sets_list)

def combine_LDA_sets(sets_list):
    # Create an empty dictionary to store the combined values
    combined_values = {}

    # Iterate over each set in the list
    for s in sets_list:
        # Iterate over each key-value pair in the set
        for key, value in s:
            # Check if the key already exists in the dictionary
            if key in combined_values:
                # Add the value to the existing key
                combined_values[key] = torch.stack([combined_values[key] , value])
            else:
                # Add a new key-value pair to the dictionary
                combined_values[key] = value
    # Create a set from the combined key-value pairs
    # combined_set = set(combined_values.items())

    for key , val in combined_values.items():
        combined_values[key] = np.mean(val)
        
    return combined_values

########## GET LDA 

def get_LDA_aspects(df , all_df):

    df['LDA_res'] = ''
    df['LDA_loc'] = ''

    for idx , row in df.iterrows():
        if idx % 1000 ==0 :
            print(f'Now progress... {idx}')
        res = all_df[all_df.name == df.name[idx]]
        res = res[res.postal_code != row.postal_code]
        loc = all_df[all_df.postal_code == row.postal_code]
        loc = loc[loc.name != df.name[idx]]
        
        res_sets_list = list(res.LDA_aspects)
        loc_sets_list = list(loc.LDA_aspects)
        
        df['LDA_res'][idx] = combine_sets(res_sets_list)
        df['LDA_loc'][idx] = combine_sets(loc_sets_list)

    return df


In [ ]:
train_df = get_LDA_aspects(train_df , all_df)
test_df = get_LDA_aspects(test_df , all_df)

In [ ]:
# Example list of sets
# sets_list = list(train_df[:5].LDA_res)
# combine_sets(sets_list)

def combine_senti_sets(sets_list):
    # Create an empty dictionary to store the combined values
    combined_values = {}
    # Iterate over each set in the list
    for s in sets_list:
        # Iterate over each key-value pair in the set
        for key, value in s.items():
            
            # Check if the key already exists in the dictionary
            if key in combined_values:
                combined_values[key].append(value)
            else:
                # Add a new key-value pair to the dictionary
                combined_values[key] = []
                combined_values[key].append(value)
    # Create a set from the combined key-value pairs
    # combined_set = set(combined_values.items())

    for key , val in combined_values.items():
        combined_values[key] =  torch.mean(torch.stack(val), dim=0)
    return combined_values

########## GET LDA 

def get_LDA_senti(df , all_df):

    df['LDA_res_senti'] = ''
    df['LDA_loc_senti'] = ''

    for idx , row in df.iterrows():
        if idx % 1000 ==0 :
            print(f'Now progress... {idx}')
        res = all_df[all_df.name == df.name[idx]]
        res = res[res.postal_code != row.postal_code]
        loc = all_df[all_df.postal_code == row.postal_code]
        loc = loc[loc.name != df.name[idx]]
        
        res_sets_list = list(res.LDA_senti)
        loc_sets_list = list(loc.LDA_senti)
        
        df['LDA_res_senti'][idx] = combine_senti_sets(res_sets_list)
        df['LDA_loc_senti'][idx] = combine_senti_sets(loc_sets_list)

    return df


In [ ]:
train_df = get_LDA_senti(train_df , all_df)
test_df = get_LDA_senti(test_df , all_df)

In [102]:
def combine_bert_embedding(df , all_df , col_name):
    
    df['res_'+str(col_name)] = ''
    df['loc_'+str(col_name)] = ''
    
    for idx , row in df.iterrows():
        
        if idx % 1000 ==0 :
            print(f'Now progress... {idx}')
        
        res = all_df[all_df.name == df.name[idx]]
        res = res[res.postal_code != row.postal_code]
        loc = all_df[all_df.postal_code == row.postal_code]
        loc = loc[loc.name != df.name[idx]]

        res_embedding = list(res[col_name])
        loc_embedding = list(loc[col_name])
        res_combined = []
        loc_combined = []
        for res_emb in res_embedding:
            res_combined.append(torch.mean(torch.stack(res_emb[0]), dim=0))
        for loc_emb in loc_embedding:
            loc_combined.append(torch.mean(torch.stack(loc_emb[0]), dim=0))
        df['res_'+str(col_name)][idx] = res_combined
        df['loc_'+str(col_name)][idx] = loc_combined
        
    return df

In [91]:
all_df.columns

Index(['business_id', 'stars_x', 'useful', 'funny', 'cool', 'text', 'date',
       'name', 'address', 'city', 'state', 'postal_code', 'latitude',
       'longitude', 'stars_y', 'review_count', 'is_open', 'attributes',
       'categories', 'hours', 'LDA_aspects', 'BERT_embedding', 'LDA_senti',
       'self_senti'],
      dtype='object')

In [103]:
print(combine_bert_embedding(train_df[:1] , all_df ,'BERT_embedding'))

Now progress... 0


/mnt/data/emma/bilab/Steph_C/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/mnt/data/emma/bilab/Steph_C/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [ ]:
train_df = combine_bert_embedding(train_df , all_df)
test_df = combine_bert_embedding(test_df , all_df)

In [ ]:
all_df.head()

In [ ]:
def combine_senti(df):
    
    df['res_emb'] = ''
    df['loc_emb'] = ''
    
    for idx , row in df.iterrows():
        
        if idx % 1000 ==0 :
            print(f'Now progress... {idx}')
        
        res = all_df[all_df.name == df.name[idx]]
        res = res[res.postal_code != row.postal_code]
        loc = all_df[all_df.postal_code == row.postal_code]
        loc = loc[loc.name != df.name[idx]]
        
        res_embedding = list(res.BERT_embedding)
        loc_embedding = list(loc.BERT_embedding)
        df['res_emb'][idx] = torch.mean(torch.stack(res_embedding), dim=0)
        df['loc_emb'][idx] = torch.mean(torch.stack(loc_embedding), dim=0)
        
        
    return df
    

## Affinity and Complementary

In [ ]:
def get_affinity_complementary(df):
    df['affinity'] = ''
    df['complementary'] = ''
    
    for idx , row in df.iterrows():
        affinity , complementary = 0,0
        if idx % 1000 ==0 :
            print(f'Now progress... {idx}')
        for res_key , res_val in df.LDA_res[idx].items():
            if res_key in df.LDA_loc[0]:
                affinity += res_val*train_df.LDA_loc[0][res_key]
                complementary += res_val*(1-train_df.LDA_loc[0][res_key])
        df['affinity'][idx] = affinity
        df['complementary'][idx] = complementary
        
    return df

In [ ]:
train_df = get_affinity_complementary(train_df)
test_df = get_affinity_complementary(test_df)

In [ ]:
import torch.nn.functional as F

def get_cosine_similarity(df):
    df['cosine_sim'] = ''
    
    for idx , row in df.iterrows():
        if idx % 1000 ==0 :
            print(f'Now progress... {idx}')
        df['cosine_sim'][idx]= F.cosine_similarity(row.res_emb.unsqueeze(0), row.loc_emb.unsqueeze(0))
    return df

In [ ]:
train_df = get_cosine_similarity(train_df)
test_df = get_cosine_similarity(test_df)

In [ ]:
def get_LDA_senti_score(df):
    
    df['LDA_senti_score'] = ''
    pos_neg_list = [-1,0,1]
    
    for idx , row in df.iterrows():
        senti_score = 0
        if idx % 1000 ==0 :
            print(f'Now progress... {idx}')
        for res_key , res_val in df.LDA_res[idx].items():
            if res_key in df.LDA_loc[0]:
                res_sentiment = pos_neg_list[np.argmax(df.LDA_senti[idx][res_key])]
                affinity += res_val**train_df.LDA_loc[0][res_key]
                complementary += res_val*(1-train_df.LDA_loc[0][res_key])
        df['affinity'][idx] = affinity
        df['complementary'][idx] = complementary
        
    return df
    for idx , row in df.iterrows:
        if idx % 1000 ==0:
            print(f'Now progress ... {idx}')
        for res_key , res_val in df.LDA_res[idx].items():
            
    return df

In [ ]:
def get_self_senti_score(df):
    
    df['self_senti_score'] = ''
    pos_neg_list = [-1,0,1]
    
    for idx , row in df.iterrows():
        senti_score = 0
        if idx % 1000 ==0 :
            print(f'Now progress... {idx}')
        for res_key , res_val in df.LDA_res[idx].items():
            if res_key in df.LDA_loc[0]:
                affinity += res_val*train_df.LDA_loc[0][res_key]
                complementary += res_val*(1-train_df.LDA_loc[0][res_key])
        df['affinity'][idx] = affinity
        df['complementary'][idx] = complementary
        
    return df
    for idx , row in df.iterrows:
        if idx % 1000 ==0:
            print(f'Now progress ... {idx}')
        for res_key , res_val in df.LDA_res[idx].items():
            
    return df

In [ ]:
train_df.to_pickle('../data/Train_by_postoal_code_pointwise_v3_4.pkl')
test_df.to_pickle('../data/Test_by_postoal_code_pointwise_v3_4.pkl')